In [1]:
import sys
sys.path.append("/kaggle/input/marigold_model/transformers/default/2/marigold_source")
from Marigold.marigold.marigold_depth_pipeline import MarigoldDepthPipeline

2025-08-14 19:22:02.567284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755199322.742393      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755199322.788460      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os, gc, cv2, torch, shutil
import numpy as np
from PIL import Image

# ========== 1) 模型加载 ==========
model_path = "/kaggle/input/marigold_model/transformers/default/2/marigold_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = MarigoldDepthPipeline.from_pretrained(model_path).to(device)

# ========== 2) 路径 ==========
base_dir = "/kaggle/input/kitti-dataset/depth_selection/val_selection_cropped"
img_dir = os.path.join(base_dir, "image")
gt_dir  = os.path.join(base_dir, "groundtruth_depth")

OUT_ROOT = "/kaggle/working/download/kitti_marigold"
for d in ["pred_raw_npy","pred_aligned_npy","gt_npy","gt_png16"]:
    os.makedirs(os.path.join(OUT_ROOT, d), exist_ok=True)

# ========== 3) 工具 ==========
def npy_save(path, arr): 
    os.makedirs(os.path.dirname(path), exist_ok=True)
    np.save(path, arr.astype(np.float32))

def mm_png_save(path, depth_m):
    """保存 GT 为 16-bit PNG（毫米）"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    depth_mm = np.clip(depth_m * 1000.0, 0, 65535).astype(np.uint16)
    cv2.imwrite(path, depth_mm)

def scale_match(pred, gt, mask):
    """每张图纯缩放对齐（scale-only）"""
    v = mask & np.isfinite(gt) & np.isfinite(pred) & (gt > 0)
    if not np.any(v): 
        return 1.0
    p, g = pred[v], gt[v]
    den = float((p**2).sum())
    return float((g * p).sum() / den) if den > 0 else 1.0

# ========== 4) 主循环：仅保存 预测 + GT ==========
img_files = sorted(f for f in os.listdir(img_dir) if f.endswith(".png"))
saved = 0

for img_file in img_files:
    # 关键：GT 文件名映射（只替换第一个 "_image_"）
    gt_file = img_file.replace("_image_", "_groundtruth_depth_", 1)
    rgb_path = os.path.join(img_dir, img_file)
    gt_path  = os.path.join(gt_dir,  gt_file)
    if not os.path.exists(gt_path):
        continue

    # 读取 RGB / GT（KITTI 16-bit PNG → 米）
    rgb_bgr = cv2.imread(rgb_path)
    if rgb_bgr is None:
        continue
    rgb = cv2.cvtColor(rgb_bgr, cv2.COLOR_BGR2RGB)

    gt_png = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED)
    if gt_png is None:
        continue
    gt_depth = gt_png.astype(np.float32) / 256.0
    mask = gt_depth > 0

    # —— 推理（Marigold 相对深度） ——
    with torch.no_grad():
        result = pipe(
            Image.fromarray(rgb),
            denoising_steps=4,
            ensemble_size=1,
            processing_res=768,
            match_input_res=True,
            batch_size=0,
            show_progress_bar=False,
        )
    pred_rel = np.asarray(result.depth_np, dtype=np.float32)

    # 尺寸对齐到 GT
    if pred_rel.shape != gt_depth.shape:
        pred_rel = cv2.resize(pred_rel, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

    # 可选：scale-only 对齐一份（便于快速对比；统一协议评测时用原始 + alignment="ls"）
    s = scale_match(pred_rel, gt_depth, mask)
    pred_aligned = pred_rel * s

    stem = os.path.splitext(img_file)[0]

    # 保存预测（相对深度 .npy；以及对齐后的 .npy）
    npy_save(os.path.join(OUT_ROOT, "pred_raw_npy",     f"{stem}.npy"), pred_rel)
    npy_save(os.path.join(OUT_ROOT, "pred_aligned_npy", f"{stem}.npy"), pred_aligned)

    # 保存 GT（米制 .npy + 16-bit 毫米 PNG）
    npy_save(os.path.join(OUT_ROOT, "gt_npy",     f"{stem}.npy"), gt_depth)
    mm_png_save(os.path.join(OUT_ROOT, "gt_png16", f"{stem}.png"), gt_depth)

    saved += 1

    # 清理
    del rgb_bgr, rgb, gt_depth, pred_rel, pred_aligned, result
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print(f"[KITTI Marigold SAVE] saved pairs: {saved} / {len(img_files)}")

# ========== 5) 打包下载 ==========
zip_base = "/kaggle/working/marigold_kitti_preds_and_gt"
shutil.make_archive(zip_base, "zip", OUT_ROOT)
print(f"打包完成: {zip_base}.zip")


The config attributes {'prediction_type': 'depth'} were passed to MarigoldDepthPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'prediction_type': 'depth'} are not expected by MarigoldDepthPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

[KITTI Marigold SAVE] saved pairs: 1000 / 1000
打包完成: /kaggle/working/marigold_kitti_preds_and_gt.zip
